In [ ]:
import os, json, tarfile
from mpcontribs.client import load_client
from pymatgen import Structure, MPRester
from urllib.request import urlretrieve
from monty.json import MontyDecoder

In [ ]:
project = 'jarvis_dft'
client = load_client('your-api-key-here')
mpr = MPRester()

Create project (once)

In [ ]:
is_public = True
info = {
    'project': project,
    'is_public': is_public,
    'title': 'JARVIS-DFT',
    'owner': 'kamal.choudhary@nist.gov',
    'authors': 'K. Choudhary, F. Tavazza',
    'description': ' '.join('''
        The DFT section of JARVIS (JARVIS-DFT) consists of thousands
        of VASP based calculations for 3D-bulk, single layer (2D), nanowire (1D) and 
        molecular (0D) systems. Most of the calculations are carried out with optB88vDW functional.
        JARVIS-DFT includes materials data such as: energetics, diffraction pattern, radial distribution
        function, band-structure, density of states, carrier effective mass, temperature and carrier
        concentration dependent thermoelectric properties, elastic constants and gamma-point phonons.
    '''.replace('\n', '').split()),
    'urls': {
        'JARVIS': 'http://www.ctcms.nist.gov/~knc6/JVASP.html',
        'SciRep': 'https://doi.org/10.1038/s41598-017-05402-0',
        'GitHub': 'https://github.com/usnistgov/jarvis'
    }
}
# client.projects.create_entry(project=info).result()

Retrieve and update project info

In [ ]:
client.projects.get_entry(pk=project, _fields=['_all']).result()

In [ ]:
client.projects.update_entry(pk=project, project={
    'long_title': '0-3D DFT Calculations by NIST',
    'description': info['description']
}).result()

Source data

In [ ]:
config = {
    "files": [
        "https://www.ctcms.nist.gov/~knc6/jdft_{}.json.tgz".format(t)
        for t in ['3d', '2d']
    ],
    "details": "https://www.ctcms.nist.gov/~knc6/jsmol/{}.html",
    'columns': {  # 'mpid'
        'jid': {'name': 'details'},
        'exfoliation_en': {'name': 'Eₓ', 'unit': 'eV'},
        'form_enp': {'name': 'ΔH', 'unit': 'eV'},
        'fin_en': {'name': 'E', 'unit': 'meV'},
        'op_gap': {'name': 'ΔE|vdW', 'unit': 'meV'},
        'mbj_gap': {'name': 'ΔE|mbj', 'unit': 'meV'},
        'kv': {'name': 'Kᵥ', 'unit': 'GPa'},
        'gv': {'name': 'Gᵥ', 'unit': 'GPa'},
        'magmom': {'name': 'μ', 'unit': 'μᵇ'}
    }
}

In [ ]:
raw_data = {}  # as read from raw files
for url in config['files']:
    dbfile = url.rsplit('/')[-1]
    if not os.path.exists(dbfile):
        print('downloading', dbfile, '...')
        urlretrieve(url, dbfile)

    print(dbfile, 'loading ...')
    with tarfile.open(dbfile, "r:gz") as tar:
        member = tar.getmembers()[0]
        raw_data[dbfile] = json.load(tar.extractfile(member), cls=MontyDecoder)
        
    print(dbfile, len(raw_data[dbfile]))

3D (bulk) materials

In [ ]:
fn = 'jdft_3d.json.tgz'
for idx, rd in enumerate(raw_data[fn]):
    identifier = rd['mpid']
    print(idx, identifier, rd['jid'])
    contrib = {'project': project, 'identifier': identifier, 'is_public': is_public, 'data': {}}
    for k, col in config['columns'].items():
        hdr, unit = col['name'], col.get('unit')
        if k == 'jid':
            contrib['data'][hdr] = config[hdr].format(rd[k])
        elif k in rd:
            if unit:
                try:
                    float(rd[k])
                except ValueError:
                    continue
            contrib['data'][hdr] = f'{rd[k]} {unit}' if unit else rd[k]
            
    ntries = 0
    while ntries < 3:
        try:
            for d in client.contributions.get_entries(
                project=project, identifier=identifier, _fields=['id']
            ).result()['data']:
                client.contributions.delete_entry(pk=d['id']).result()
                print(idx, d['id'], 'deleted')

            cid = client.contributions.create_entry(contribution=contrib).result()['id']
            #print(client.contributions.get_entry(pk=cid, _fields=['_all']).result())
            print(idx, cid, 'created')
            
            structure = rd['final_str']
            comp = structure.composition.reduced_formula
            sdct = {'contribution': cid, 'label': '02/24/2020', 'name': comp, 'is_public': is_public}
            sdct.update(structure.as_dict())
            sid = client.structures.create_entry(structure=sdct).result()['id']
            print(idx, 'structure', sid, 'created')
            break
        except Exception as ex:
            ntries += 1
            print(ntries, ex)
            print('waiting', ntries*30, 'seconds...')
            sleep(ntries*30)
    else:
        print('I give up.')
        break
        
    # final_str
    print(contrib)
    break
    